In [ ]:
!pip install datasets
!pip install transformers[torch]
!pip install tensorflow -U

In [24]:
torch.cuda.is_available()

False

In [ ]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments
from transformers import DataCollatorWithPadding
from sklearn.model_selection import train_test_split
from datasets import load_dataset
import torch

# Tokenizer
tokenizer = AutoTokenizer.from_pretrained('bert-base-uncased')
# Model
model = AutoModelForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=2)
model.to('cuda' if torch.cuda.is_available() else 'cpu')

# model to device

dataset = load_dataset("imdb")
def tokenize(batch):
    return tokenizer(batch["text"], padding=True, truncation=True)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
train_dataset = dataset["train"].select(range(25)).map(tokenize, batched=True)


In [27]:
config = {
 "tests": {
  "defaults": {
   "min_pass_rate": 1.0
  },
  "robustness": {
   "add_typo": {"min_pass_rate": 0.7},
   "uppercase": {"min_pass_rate": 0.7},
   "american_to_british": {"min_pass_rate": 0.7},
  },
  "accuracy": {
   "min_micro_f1_score": {
    "min_score": 0.7
   }
  },
  "bias": {
   "replace_to_female_pronouns": {
    "min_pass_rate": 0.7
   },
   "replace_to_low_income_country": {
    "min_pass_rate": 0.7
   }
  }
 }
}

In [28]:
my_callback = LangTestCallback(task="text-classification", data={"data_source":"sample.csv"}, config=config, save_reports=True, run_each_epoch=True)

In [29]:
# Training arguments
training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=3,
)

trainer = Trainer(
    model=model,
    tokenizer=tokenizer,
    args=training_args,
    train_dataset=train_dataset,
    callbacks=[my_callback]
)

Test Configuration : 
 {
 "tests": {
  "defaults": {
   "min_pass_rate": 1.0
  },
  "robustness": {
   "add_typo": {
    "min_pass_rate": 0.7
   },
   "uppercase": {
    "min_pass_rate": 0.7
   },
   "american_to_british": {
    "min_pass_rate": 0.7
   }
  },
  "accuracy": {
   "min_micro_f1_score": {
    "min_score": 0.7
   }
  },
  "bias": {
   "replace_to_female_pronouns": {
    "min_pass_rate": 0.7
   },
   "replace_to_low_income_country": {
    "min_pass_rate": 0.7
   }
  }
 }
}


In [30]:
# Training the model
trainer.train()

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:2649: FutureWarning: The `truncation_strategy` argument is deprecated and will be removed in a future version, use `truncation=True` to truncate examples to a max length. You can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to truncate to the maximal input size of the model (e.g. 512 for Bert).  If you have pairs of inputs, you can give a specific truncation strategy selected among `truncation='only_first'` (will only truncate the first sentence in the pairs) `truncation='only_second'` (will only truncate the second sentence in the pairs) or `truncation='longest_first'` (will iteratively remove tokens from the longest sentence in the pairs).
  warnings.warn(
Generating testcases...: 100%|██████████| 3/3 [00:00<00:00, 11533.37it/s]
[W010] - Test 'add_typo': 2 samples removed out of 18
[W010] - Test 'american_to_british': 9 samples removed out of 18

[W010] - Tes

Step,Training Loss


Running testcases... : 100%|██████████| 64/64 [00:17<00:00,  3.63it/s]


     category                      test_type  fail_count  pass_count  \
0  robustness                       add_typo           0          16   
1  robustness                      uppercase           0          18   
2  robustness            american_to_british           0           9   
3    accuracy             min_micro_f1_score           1           0   
4        bias     replace_to_female_pronouns           0          16   
5        bias  replace_to_low_income_country           0           4   

  pass_rate minimum_pass_rate   pass  
0      100%               70%   True  
1      100%               70%   True  
2      100%               70%   True  
3        0%              100%  False  
4      100%               70%   True  
5      100%               70%   True  


Running testcases... :   0%|          | 0/64 [00:00<?, ?it/s]/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:2649: FutureWarning: The `truncation_strategy` argument is deprecated and will be removed in a future version, use `truncation=True` to truncate examples to a max length. You can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to truncate to the maximal input size of the model (e.g. 512 for Bert).  If you have pairs of inputs, you can give a specific truncation strategy selected among `truncation='only_first'` (will only truncate the first sentence in the pairs) `truncation='only_second'` (will only truncate the second sentence in the pairs) or `truncation='longest_first'` (will iteratively remove tokens from the longest sentence in the pairs).
  warnings.warn(
Running testcases... : 100%|██████████| 64/64 [00:02<00:00, 31.00it/s]


     category                      test_type  fail_count  pass_count  \
0  robustness                       add_typo           0          16   
1  robustness                      uppercase           0          18   
2  robustness            american_to_british           0           9   
3    accuracy             min_micro_f1_score           1           0   
4        bias     replace_to_female_pronouns           0          16   
5        bias  replace_to_low_income_country           0           4   

  pass_rate minimum_pass_rate   pass  
0      100%               70%   True  
1      100%               70%   True  
2      100%               70%   True  
3        0%              100%  False  
4      100%               70%   True  
5      100%               70%   True  


Running testcases... :   0%|          | 0/64 [00:00<?, ?it/s]/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:2649: FutureWarning: The `truncation_strategy` argument is deprecated and will be removed in a future version, use `truncation=True` to truncate examples to a max length. You can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to truncate to the maximal input size of the model (e.g. 512 for Bert).  If you have pairs of inputs, you can give a specific truncation strategy selected among `truncation='only_first'` (will only truncate the first sentence in the pairs) `truncation='only_second'` (will only truncate the second sentence in the pairs) or `truncation='longest_first'` (will iteratively remove tokens from the longest sentence in the pairs).
  warnings.warn(
Running testcases... : 100%|██████████| 64/64 [00:02<00:00, 31.19it/s]

     category                      test_type  fail_count  pass_count  \
0  robustness                       add_typo           0          16   
1  robustness                      uppercase           0          18   
2  robustness            american_to_british           0           9   
3    accuracy             min_micro_f1_score           1           0   
4        bias     replace_to_female_pronouns           0          16   
5        bias  replace_to_low_income_country           0           4   

  pass_rate minimum_pass_rate   pass  
0      100%               70%   True  
1      100%               70%   True  
2      100%               70%   True  
3        0%              100%  False  
4      100%               70%   True  
5      100%               70%   True  


TrainOutput(global_step=12, training_loss=0.0014002219152947266, metrics={'train_runtime': 67.7653, 'train_samples_per_second': 1.107, 'train_steps_per_second': 0.177, 'total_flos': 19733329152000.0, 'train_loss': 0.0014002219152947266, 'epoch': 3.0})

In [ ]:
!pip install langtest

In [ ]:
import os
from transformers import TrainerCallback, pipeline
from langtest import Harness


class LangTestCallback(TrainerCallback):
    def __init__(
        self,
        task,
        config=None,
        data=None,
        print_reports=True,
        save_reports=False,
        run_each_epoch=False,
    ) -> None:
        super().__init__()
        self.task = task
        self.config = config
        self.data = data
        self.print_reports = print_reports
        self.save_reports = save_reports
        self.run_each_epoch = run_each_epoch
        self.epoch_number = 0
        self.harness = None

    def on_init_end(self, args, state, control, **kwargs):
        model = kwargs["model"]
        tokenizer = kwargs["tokenizer"]

        if self.task == "ner":
            model = pipeline("ner", model=model, tokenizer=tokenizer, ignore_labels=[])
        elif self.task == "text-classification":
            model = pipeline("text-classification", model=model, tokenizer=tokenizer)

        self.harness = Harness(
            self.task,
            {
                "model": model,
                "hub": "huggingface",
            },
            self.data,
            self.config,
        )

    def on_epoch_end(self, args, state, control, **kwargs):
        self.epoch_number += 1

        if self.run_each_epoch:
            self.harness._generated_results = None
            self.harness.run()
            if self.print_reports:
                print(self.harness.report())

            if self.save_reports:
                if not os.path.exists("reports"):
                    os.mkdir("reports")
                self.harness.report(
                    format="markdown",
                    save_dir=f"reports/report{self.epoch_number:03d}.md",
                )

    def on_train_begin(self, args, state, control, **kwargs):
        self.harness.generate()

    def on_train_end(self, args, state, control, **kwargs):
        if self.run_each_epoch:
            return

        print(self.harness.run().report())
        if self.save_reports:
            if not os.path.exists("reports"):
                os.mkdir("reports")
            self.harness.report(
                format="markdown", save_dir=f"reports/report{self.epoch_number:03d}.md"
            )


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 475.2/475.2 MB 3.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 115.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 442.0/442.0 kB 41.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 87.9 MB/s eta 0:00:00
  Attempting uninstall: tensorflow-estimator
    Found existing installation: tensorflow-estimator 2.12.0
    Uninstalling tensorflow-estimator-2.12.0:
      Successfully uninstalled tensorflow-estimator-2.12.0
  Attempting uninstall: keras
    Found existing installation: keras 2.12.0
    Uninstalling keras-2.12.0:
      Successfully uninstalled keras-2.12.0
  Attempting uninstall: google-auth-oauthlib
    Found existing installation: google-auth-oauthlib 0.4.6
    Uninstalling google-auth-oauthlib-0.4.6:
      Successfully uninstalled google-auth-oauthlib-0.4.6
  Attempting uninstall: tensorboard
    Found existing installation: tensorboard 2.12.0
    Uninstalling t